**Install Dependencies**

In [2]:
!pip install transformers pandas gradio openpyxl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 88.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 103.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 11.3 MB/s eta 0:00:00


 **Import Libraries**

In [3]:
import pandas as pd
import gradio as gr
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer

**Load The Model**

In [4]:
# Load an open-source language model (GPT-2 for demonstration)
model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Create the conversational pipeline
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

**Function to Load Excel Data**
* This function will allow the user to upload an Excel file and read it into a pandas DataFrame

In [5]:
uploaded_data = None

def load_excel(file):
    global uploaded_data
    uploaded_data = pd.read_excel(file)
    return "Excel file loaded successfully!"

**Create Chatbot Function**
* The chatbot will process user queries and provide insights based on the uploaded data

In [6]:
def chatbot(user_input):
    global uploaded_data

    if uploaded_data is None:
        return "Please upload an Excel file before asking questions."

    # Check if the user is asking about the dataset
    if 'dataset' in user_input or 'data' in user_input:
        if 'column' in user_input or 'columns' in user_input:
            return f"The dataset has the following columns: {', '.join(uploaded_data.columns)}"
        elif 'shape' in user_input:
            return f"The dataset has {uploaded_data.shape[0]} rows and {uploaded_data.shape[1]} columns."
        elif 'summary' in user_input:
            return f"Here is a summary of the dataset:\n{uploaded_data.describe()}"
        else:
            return "I can provide insights such as the column names, dataset shape, or summary statistics."

    # If the question is more general, use the language model to generate an answer
    response = generator(user_input, max_length=100, num_return_sequences=1)
    return response[0]['generated_text']


**Gradio Interface**
* Use Gradio to create an easy-to-use interface

In [7]:
# Gradio interface for uploading Excel and asking questions
def interface():
    with gr.Blocks() as demo:
        # Upload Excel file
        excel_input = gr.File(label="Upload Excel file", file_types=[".xlsx"])
        excel_output = gr.Textbox(label="File Upload Status")

        # User query input
        user_input = gr.Textbox(label="Ask a question about the dataset or any general query")
        chatbot_output = gr.Textbox(label="Chatbot Response")

        # Define the logic
        excel_input.upload(load_excel, inputs=excel_input, outputs=excel_output)
        user_input.submit(chatbot, inputs=user_input, outputs=chatbot_output)

    demo.launch()

# Run the interface
interface()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://89f05b0688d2d6eb3c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
